## PBL(1): Individual Project Work

1. https://agtechresearch.pythonanywhere.com/ 에 접속하여 회원가입해 주세요. (비밀번호는 단순하게 만드는 것을 권장. 예: 1234)
2. `username` 에 이메일 형식의 아이디를 기입해 주세요.
3. `password` 에 비밀번호를 기입해 주세요.

## PBL(1): Individual Project Work

1. https://agtechresearch.pythonanywhere.com/ 에 접속하여 회원가입해 주세요. (비밀번호는 단순하게 만드는 것을 권장. 예: 1234)
2. `username` 에 이메일 형식의 아이디를 기입해 주세요.
3. `password` 에 비밀번호를 기입해 주세요.

In [8]:
project = "housingprice"  # 수정하지 마세요
username = "g4312@naver.com"  # 회원가입 시 사용한 이메일아이디 (예시. abc@hello.com)
password = "1234"  # 비밀번호


리더보드 제출을 위한 기본 설정: 아래 코드를 실행해주세요.


In [9]:
import os
import urllib.request

if not os.path.exists("competition.py"):
    url = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/competition/competition.py"
    filename = "competition.py"
    urllib.request.urlretrieve(url, filename)

아래 코드를 실행하여 데이터를 다운로드 받습니다: 3개의 csv 파일이 data 폴더에 다운로드됨

 * dataset.csv: 과거 주택매매 데이터 -> 학습에 사용할 데이터셋
 * problem.csv: 현재 A사가 매매를 고려하고 있는 130건의 주택정보 -> ML 모델에 의하여 예측을 수행하여야 할 데이터셋
 * submission.csv: 리더보드 서버 제출을 위한 파일 형식


In [10]:
import competition

# 파일 다운로드
competition.download_competition_files(project)

100%|██████████| 141k/141k [00:00<00:00, 338kiB/s] 


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# Data 경로 설정
DATA_DIR = "data"

In [12]:
# 메모리를 효율적으로 사용하기 위한 downcast 함수 정의
def downcast(df, verbose=True):     # verbose 옵션 추가: (True)인 경우 몇 퍼센트 압축됐는지 출력
    start_mem = df.memory_usage().sum() / 1024**2   # 초기 메모리 사용량
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print(f'{(100*(start_mem - end_mem) / start_mem):.1f}% 압축됨')

    return df

In [13]:
# 학습에 사용할 과거 주택매매 data set 로드 (dataset.csv)
dataset = pd.read_csv(os.path.join(DATA_DIR, "dataset.csv"))

# problem set 로드 (problem.csv)
problemset = pd.read_csv(os.path.join(DATA_DIR, "problem.csv"))

In [14]:
downcast(dataset)
downcast(problemset)

67.2% 압축됨
68.7% 압축됨


,Id,LotArea,Street,LotConfig,OverallQual,OverallCond,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,...,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageType,GarageYrBlt,GarageCars,GarageArea,YrSold
0,1341,11200,Pave,Inside,5,5,1965,1965,1040,1040,...,0,3,1,5,0,Detchd,1965,1,384,2008
1,1342,7200,Pave,Corner,5,7,1951,2000,900,900,...,0,3,1,5,0,Detchd,2005,2,576,2010
2,1343,16905,Pave,Inside,5,6,1959,1959,1350,1328,...,1,2,1,5,2,Attchd,1959,1,308,2007
3,1344,9180,Pave,CulDSac,5,7,1983,1983,840,884,...,0,2,1,5,0,Attchd,1983,2,504,2007
4,1345,7200,Pave,Inside,5,7,1920,1996,530,581,...,0,3,1,6,0,Detchd,1935,1,288,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,1466,11478,Pave,Inside,8,5,2007,2008,1704,1704,...,0,3,1,7,1,Attchd,2008,3,772,2010
126,1467,16321,Pave,CulDSac,5,6,1957,1997,1484,1600,...,0,2,1,6,2,Attchd,1957,1,319,2006
127,1468,6324,Pave,Inside,4,6,1927,1950,520,520,...,0,1,1,4,0,Detchd,1920,1,240,2008
128,1469,8500,Pave,Inside,4,4,1920,1950,649,649,...,0,3,1,6,0,Detchd,1920,1,250,2008


In [15]:
# # 결측치가 있다면 GarageType은 NO Garage로, GarageYrBlt의 값은 0으로 대체
dataset["GarageType"].fillna("No Garage", inplace=True)
dataset["GarageYrBlt"].fillna(0, inplace=True)

dataset.isnull().sum()


Id              0
LotArea         0
Street          0
LotConfig       0
OverallQual     0
OverallCond     0
YearBuilt       0
YearRemodAdd    0
TotalBsmtSF     0
1stFlrSF        0
2ndFlrSF        0
GrLivArea       0
FullBath        0
HalfBath        0
BedroomAbvGr    0
KitchenAbvGr    0
TotRmsAbvGrd    0
Fireplaces      0
GarageType      0
GarageYrBlt     0
GarageCars      0
GarageArea      0
YrSold          0
SalePrice       0
dtype: int64

In [16]:
#FullBath와 HalfBath를 합쳐서 Bath로 만든다
#YearBuilt는 제거하고, 2024를 YearRemodAdd뺀 Year 열 추가, 이후 YearRemodAdd는 제거
#총 면적을 나타내기 위해 1stFlrSF, 2ndFlrSF, TotalBsmtSF를 합쳐서 TotalFlr로 나타낸다. 나머지는 지운다
#OverallCond제거, Id제거, KitchenAbvGr제거, YrSold제거

dataset["Bath"] = dataset["FullBath"] + dataset["HalfBath"]
dataset["Year"] = 2024 - dataset["YearRemodAdd"]
dataset["TotalFlr"] = dataset["1stFlrSF"] + dataset["2ndFlrSF"] + dataset["TotalBsmtSF"]

#이후 필요없는 열 제거
dataset.drop(["FullBath", "HalfBath", "YearBuilt", "YearRemodAdd", "1stFlrSF", "2ndFlrSF", "TotalBsmtSF",  "Id", "KitchenAbvGr", "YrSold",'Street'], axis=1, inplace=True)

# # problem set 전처리
# # problem set에도 동일한 전처리를 적용한다.
problemset["Bath"] = problemset["FullBath"] + problemset["HalfBath"]
problemset["Year"] = 2024 - problemset["YearRemodAdd"]
problemset["TotalFlr"] = problemset["1stFlrSF"] + problemset["2ndFlrSF"] + problemset["TotalBsmtSF"]
#이후 필요없는 열 제거
problemset.drop(["FullBath", "HalfBath", "YearBuilt", "YearRemodAdd", "1stFlrSF", "2ndFlrSF", "TotalBsmtSF", "Id", "KitchenAbvGr", "YrSold",'Street'], axis=1, inplace=True)


In [17]:
dataset.shape , problemset.shape

((1340, 16), (130, 15))

In [18]:
# GrLIvArea가 4000이상인 이상치 4000으로 대체
dataset.loc[dataset["GrLivArea"] > 4000, "GrLivArea"] = 4000
# LotArea가 100000이상인 이상치 100000으로 대체
dataset.loc[dataset["LotArea"] > 100000, "LotArea"] = 100000


In [19]:
# GarageYrBlt가 0이 아닐시 2024에서 뺀 값으로 대체
dataset.loc[dataset["GarageYrBlt"] != 0, "GarageYrBlt"] = 2024 - dataset["GarageYrBlt"]
# problem set에도 동일한 전처리를 적용한다.
problemset.loc[problemset["GarageYrBlt"] != 0, "GarageYrBlt"] = 2024 - problemset["GarageYrBlt"]





In [20]:
# dataset과 problemset을 합치는데, 섞이지 않게 합친다
all_data = pd.concat([dataset, problemset], ignore_index=True)
# 범주형 변수를 더미화 시킨다
all_data = pd.get_dummies(all_data)


In [21]:
# 학습 데이터와 문제 데이터로 다시 분리
dataset = all_data.iloc[:len(dataset)]
problemset = all_data.iloc[len(dataset):]

In [22]:
dataset.shape, problemset.shape

((1340, 26), (130, 26))

In [23]:
problemset.drop("SalePrice", axis=1, inplace=True)

In [24]:
dataset.shape, problemset.shape

((1340, 26), (130, 25))

In [25]:
# 데이터셋 분리
from sklearn.model_selection  import train_test_split
x_train = dataset.drop("SalePrice", axis=1)
y_train = dataset["SalePrice"]
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [26]:
# Random over-sampling 적용
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
x_train, y_train = ros.fit_resample(x_train, y_train)



In [27]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((8720, 25), (268, 25), (8720,), (268,))

In [28]:
x_problem = problemset

In [78]:
x_train.shape, x_test.shape , x_problem.shape

((8720, 25), (268, 25), (130, 25))

In [30]:
# 모델 정의
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X=x_train):
    rmse = np.sqrt(-cross_val_score(model, X, y_train, scoring="neg_mean_squared_error", cv=kfolds))
    return rmse

In [31]:
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [37]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=10000, alphas=alphas2, random_state=42, cv=kfolds))
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=10000, alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))                                
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003,))
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =42)
xgboost = XGBRegressor(learning_rate=0.01,n_estimators=3460, max_depth=3, min_child_weight=0, gamma=0, subsample=0.7, colsample_bytree=0.7, nthread=-1, scale_pos_weight=1, seed=27, reg_alpha=0.00006)
lightgbm = LGBMRegressor(objective='regression', num_leaves=4, learning_rate=0.01, n_estimators=5000, max_bin=200, bagging_fraction=0.75, bagging_freq=5, bagging_seed=7, feature_fraction=0.2, feature_fraction_seed=7, verbose=-1,)
stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

In [38]:
# 모델별 교차 검증 수행 및 결과 출력
score1 = cv_rmse(ridge,x_train)
print("Ridge: {:.4f} ({:.4f})\n".format(score1.mean(), score1.std()))

score2 = cv_rmse(lasso,x_train)
print("LASSO: {:.4f} ({:.4f})\n".format(score2.mean(), score2.std()))

score3 = cv_rmse(elasticnet)
print("Elastic net: {:.4f} ({:.4f})\n".format(score3.mean(), score3.std()))

score4 = cv_rmse(svr)
print("SVR: {:.4f} ({:.4f})\n".format(score4.mean(), score4.std()))

score5 = cv_rmse(gbr)
print("GBR: {:.4f} ({:.4f})\n".format(score5.mean(), score5.std()))

score6 = cv_rmse(xgboost)
print("XGBoost: {:.4f} ({:.4f})\n".format(score6.mean(), score6.std()))

score7 = cv_rmse(lightgbm)
print("LightGBM: {:.4f} ({:.4f})\n".format(score7.mean(), score7.std()))

Ridge: 38899.8753 (3149.4009)

LASSO: 38876.2535 (3158.1243)

Elastic net: 38876.4277 (3157.5941)

SVR: 94007.9202 (4925.3463)

GBR: 6109.0221 (1680.7943)

XGBoost: 11422.5872 (802.5508)

LightGBM: 16538.7720 (786.7379)



In [39]:
import pandas as pd

model_performances = pd.DataFrame({
    'Model': ['Ridge', 'Lasso', 'ElasticNet', 'SVR', 'GradientBoosting', 'XGBoost', 'LightGBM'],
    'Score': [score1.mean(), score2.mean(), score3.mean(), score4.mean(), score5.mean(), score6.mean(), score7.mean()],
    'Std': [score1.std(), score2.std(), score3.std(), score4.std(), score5.std(), score6.std(), score7.std()]
})
model_performances

,Model,Score,Std
0,Ridge,38899.875316,3149.400912
1,Lasso,38876.253494,3158.124282
2,ElasticNet,38876.427702,3157.594142
3,SVR,94007.920206,4925.346293
4,GradientBoosting,6109.022077,1680.794273
5,XGBoost,11422.587184,802.550823
6,LightGBM,16538.772013,786.737897


In [40]:
print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(x_train), np.array(y_train))

print('elasticnet')
elastic_model_full_data = elasticnet.fit(x_train, y_train)

print('Lasso')
lasso_model_full_data = lasso.fit(x_train, y_train)

print('Ridge')
ridge_model_full_data = ridge.fit(x_train, y_train)

print('Svr')
svr_model_full_data = svr.fit(x_train, y_train)

print('GradientBoosting')
gbr_model_full_data = gbr.fit(x_train, y_train)

print('xgboost')
xgb_model_full_data = xgboost.fit(x_train, y_train)

print('lightgbm')
lgb_model_full_data = lightgbm.fit(x_train, y_train)

stack_gen
elasticnet
Lasso
Ridge
Svr
GradientBoosting
xgboost
lightgbm


In [45]:
print('Ridge RMSLE score on train data:')
print(rmsle(np.expm1(y_train), np.expm1(ridge_model_full_data.predict(x_train))))

print('Lasso RMSLE score on train data:')
print(rmsle(np.expm1(y_train), np.expm1(lasso_model_full_data.predict(x_train))))

print('ElasticNet RMSLE score on train data:')
print(rmsle(np.expm1(y_train), np.expm1(elastic_model_full_data.predict(x_train))))

print('SVR RMSLE score on train data:')
print(rmsle(np.expm1(y_train), np.expm1(svr_model_full_data.predict(x_train))))

print('GradientBoosting RMSLE score on train data:')
print(rmsle(np.expm1(y_train), np.expm1(gbr_model_full_data.predict(x_train))))

print('Xgboost RMSLE score on train data:')
print(rmsle(np.expm1(y_train), np.expm1(xgb_model_full_data.predict(x_train))))

print('lightgbm RMSLE score on train data:')
print(rmsle(np.expm1(y_train), np.expm1(lgb_model_full_data.predict(x_train))))

print('stack_gen RMSLE score on train data:')
print(rmsle(np.expm1(y_train), np.expm1(stack_gen_model.predict(np.array(x_train)))))

lightgbm RMSLE score on train data:


ValueError: Input contains infinity or a value too large for dtype('float64').

In [ ]:
def blend_models_predict(X):
    return ((0.25 * elastic_model_full_data.predict(X)) + \
            (0.2 * lasso_model_full_data.predict(X)) + \
            (0.15 * ridge_model_full_data.predict(X)) + \
            (0.2 * svr_model_full_data.predict(X)) + \
            (0.05 * gbr_model_full_data.predict(X)) + \
            (0.05 * xgb_model_full_data.predict(X)) + \
            (0.05 * lgb_model_full_data.predict(X)) + \
            (0.05 * stack_gen_model.predict(np.array(X))))

In [ ]:
print('RMSLE score on train data:')
print(rmsle(y_train, blend_models_predict(x_train)))

print('RMSLE score on train data:')
print(rmsle(np.expm1(y_train), np.expm1(blend_models_predict(x_train))))

In [82]:
# 리더보드 서버 제출을 위한 파일 생성
submission = pd.read_csv(os.path.join(DATA_DIR, "submission.csv"))
submission["SalePrice"] = (np.expm1(blend_models_predict(x_problem)))

# 예측 결과 화면에 출력 후 제출
display(submission)
competition.submit(project, username, password, submission)

,Id,SalePrice
0,1341,124766.323613
1,1342,131054.919415
2,1343,153409.016845
3,1344,134629.813537
4,1345,117156.889332
...,...,...
125,1466,300277.615547
126,1467,210204.448706
127,1468,82526.562572
128,1469,97403.749858


아이디:  g4312@naver.com
파일명:  submissions\20240514-205844-submission.csv
[제출에 성공하였습니다]
제출 결과: 23875.470763345482
